# Mutation Pipeline

In [2]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio import Datanalytics as da 
from JKBio import TerraFunction as terra
from JKBio import Helper as h
from JKBio.helper.google_sheet import GSheet
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display


%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

you need to have installed JKBio in the same folder as ccle_processing


/Users/jnoorbak/opt/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:15: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Loading BokehJS ...

## On local


### downloading from terra

In [3]:
import util.postprocess_maf as pm

In [4]:
refwm = dm.WorkspaceManager('nci-mimoun-bi-org/CCLF_WES')

In [5]:
mutation_filtered_terra_merged = pm.download_maf_from_workspace(refwm, sample_set_ids = ['all_ice', 'all_agilent'],
                                                                output_maf='/tmp/mutation_filtered_terra_merged.txt');

### postprocessing


Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

- readMutations
- createSNPs
- addToMainMutation
- filterAllelicFraction
- filterMinCoverage
- mergeAnnotations
- addAnnotation
- maf_add_variant_annotations
- mutation_maf_to_binary_matrix (x3)

#### saving samples used for 20Q2

In [7]:
%%R
source('src/CCLE_postp_function.R')
library('data.table')
newly_merged_maf <- readMutations('/tmp/mutation_filtered_terra_merged.txt')
new_release <- createSNPs(newly_merged_maf)
names(new_release)

 [1] "Hugo_Symbol"            "Entrez_Gene_Id"         "NCBI_Build"            
 [4] "Chromosome"             "Start_position"         "End_position"          
 [7] "Strand"                 "Variant_Classification" "Variant_Type"          
[10] "Reference_Allele"       "Tumor_Seq_Allele2"      "dbSNP_RS"              
[13] "dbSNP_Val_Status"       "Genome_Change"          "Annotation_Transcript" 
[16] "Tumor_Sample_Barcode"   "cDNA_Change"            "Codon_Change"          
[19] "Protein_Change"         "isDeleterious"          "isTCGAhotspot"         
[22] "TCGAhsCnt"              "isCOSMIChotspot"        "COSMIChsCnt"           
[25] "t_alt_count"            "t_ref_count"            "ExAC_AF"               
[28] "pass"                  


In [9]:
%%R
filtered <- filterAllelicFraction(new_release)

R[write to console]: Error in h(simpleError(msg, call)) : 
  error in evaluating the argument 'X' in selecting a method for function 'apply': j (the 2nd argument inside [...]) is a single symbol but column name 'ac_columns' is not found. Perhaps you intended DT[, ..ac_columns]. This difference to data.frame is deliberate and explained in FAQ 1.1.
Calls: <Anonymous> ... filterAllelicFraction -> apply -> [ -> [.data.table

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In fread(newly_merged_maf, select = c(desired_fields, additional_columns_to_keep)) :
R[write to console]: 
 
R[write to console]:  Column name 'ExAC_AF' not found in column name header (case sensitive), skipping.




Error in h(simpleError(msg, call)) : 
  error in evaluating the argument 'X' in selecting a method for function 'apply': j (the 2nd argument inside [...]) is a single symbol but column name 'ac_columns' is not found. Perhaps you intended DT[, ..ac_columns]. This difference to data.frame is deliberate and explained in FAQ 1.1.
Calls: <Anonymous> ... filterAllelicFraction -> apply -> [ -> [.data.table


In [ ]:
%%R
filtered <- filterMinCoverage(filtered$merged, filtered$removed_from_maf)

In [ ]:
%%R
clean_annotations <- mergeAnnotations(merged,previous.release.maf)

In [ ]:
%%R
# Guillaume's version
new_release <- addAnnotation(filtered$merged, clean_annotations, colnames(previous.release.maf))
# Allie's version
new_release <- maf_add_variant_annotations(new_release)

In [ ]:
def filterCoverage(maf, loc=['CGA_WES_AC'], sep=':',cov=4):
    muts=np.zeroes((len(maf),2))
    for val in loc:
        muts+= np.array([[v[0],0] if 'NA' in v else v for v in mutations_20Q2_all[val].fillna('0'+sep+'0').astype(str).str.split(sep).tolist()]).astype(int)
    return maf[muts[:,1]>=cov]

def filterAllelicFraction(maf, loc=['CGA_WES_AC'], sep=':',frac=0.3):
    muts=np.zeroes((len(maf),2))
    for val in loc:
        muts+= np.array([[v[0],0] if 'NA' in v else v for v in mutations_20Q2_all[val].fillna('0'+sep+'0').astype(str).str.split(sep).tolist()]).astype(int)
    muts = muts[:,0]/muts[:,1]
    return maf[muts>=frac]

def mergeAnnotations(newmaf, additionalmaf, additionalonmerge=[]):
    on = ['Chromosome', 'Start_position', 'End_position', 'Reference_Allele', 'Tumor_Seq_Allele1']
    on.extend(additionalonmerge)
    
    newmaf = newmaf.join(additionalmaf, on = on)
    if 
    solve issues with Hugo_Symbol, Entrez_Gene_Id
    
    
    
    return newmad
    
def mergeXY():
    dbSNP_RS.x, dbSNP_RS.y


def addAnnotation(maf, NCBI_Build='37', Strand="+"):
    maf['NCBI_Build'] = NCBI_Build
    maf['Strand'] = Strand
    maf = maf[['current', 'SangerWES_AC', 'SangerRecalibWES_AC', 'RNAseq_AC', 'HC_AC', 'RD_AC', 'WGS_AC']

def mafToMat(maf, col, boolify = False, samplesCol = "DepMap_ID", mutNameCol="Hugo_Symbol"):
    maf = maf.sort_values(by = mutNameCol)
    samples = set(maf[samplesCol])
    mut = pd.DataFrame(data = np.zeros((len(set(maf[mutNameCol])), 1)), columns=['fake'], index=set(maf[mutNameCol])).astype(float)
    for i,val in enumerate(samples):
        h.showcount(i,len(samples))
        mut = mut.join(maf[maf[samplesCol]==val].drop_duplicates(mutNameCol).set_index(mutNameCol)[col].rename(val))
    return mut.nan_to_num(0).astype(bool if boolify else float).drop(columns=['fake'])

In [ ]:
filtered_mutations = filterCoverage(mutations)
filtered_mutations = filterAllelicFraction(filtered_mutations)

merged_mutations = addAnnotation(mutations)

mafToMat(filtered_mutations[filtered_mutations.damaging]).to_csv('.csv')
mafToMat(filtered_mutations[filtered_mutations.other]).to_csv('.csv')
mafToMat(filtered_mutations[filtered_mutations.hotspot]).to_csv('.csv')


CCLE2othermutations = 

mutations = mergeAnnotations(filtered_mutations, CCLE2othermutations)

#making 
for muttype in ['']:
    mafToMat(CCLE2othermutations[CCLE2othermutations.damaging & CCLE2othermutations[muttype]]).to_csv(''+muttype+".csv")
    mafToMat(CCLE2othermutations[CCLE2othermutations.other & CCLE2othermutations[muttype]]).to_csv(''+muttype+".csv")
    mafToMat(CCLE2othermutations[CCLE2othermutations.hotspot & CCLE2othermutations[muttype]]).to_csv(''+muttype+".csv")

# uploading on taiga

In [25]:
gsheets = sheets.get(sheeturl).sheets[6].to_frame()
wes_dmc_embargo = [i for i in gsheets['WES_DMC_embargo'].values.tolist() if str(i) != "nan"]
wes_embargo = [i for i in gsheets['WES_embargo'].values.tolist() if str(i) != "nan"]
blacklist = [i for i in gsheets['blacklist'].values.tolist() if str(i) != "nan"]

In [ ]:
wes_embargo, wes_dmc_embargo, blacklist

In [ ]:
! cd .. && git clone https://github.com/broadinstitute/depmap-release-readmes.git && cd -

In [ ]:
! cd ../depmap-release-readmes && git pull

In [ ]:
!cd ../depmap-release-readmes/ && python3 make_new_release.py $release && git add . && git commit -m $release && git push 

In [ ]:
os.system('cd ../depmap-release-readmes && git pull && mv release-'+release+'/internal-'+release+'.txt ../ccle_processing/temp/README && cd -')

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutations-maf-35fe",
                 upload_file_path_dict={'temp/mutations.'+release+'.all.csv': 'TableCSV'}, 
                 dataset_description="""
# Mutations

filtered and unfiltered mutation files from Broad WES and Sanger WES data mapped to hg19
The MAF file for DepMap that includes all of the latest WES samples. This MAF is generated by merging CCLE (WGS, RNAseq, RD, HC) and Sanger (WES) data.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

version 1:  In 19Q1 the WES_AC column has been replaced by two columns, VA_WES_AC and CGA_WES_AC. We are currently using the Van Allen and CGA based pipeline to generate mutation calls. The CGA pipeline includes more filtering on the MAFs than VA and has a better INDEL caller. However, some of these filters may be removing some variants of interest that are still capture by the VA pipeline, which is why both a retained for now. DEPRECATED:  Missing the VA_WES_AC, CGA_WES_AC columns
version 2: 19Q1 data
version 3: 19Q2 data. We are no longer using the CCLE_WES_AC column. We are only using the CGA pipeline for mutation calls.
version 4: Updating to 19Q3interim DEPRECATED
version 5: Updating to 19Q3interim DEPRECATED
version 6: Updating to 19Q3interim
version 7: Updating to 19Q3 DEPRECATED
version 8: reparing the missing mutation problem DEPRECATED
version 9: reparing the missing column problem


version10:
Adding 52 new cell lines. 
Some cells lines have been flagged as:

version11:
adding missing cell lines

Adding 52 new cell lines. 
Some cells lines have been flagged as:

 - having bad looking copy ration plots = 
 - Genes having a similar CN value accross all []

version 12:

adding 8 new cell lines

version 13:

removing a wrong column

version 14:

adding 8 new cell lines. Adding .all. since we are soon going to release a restricted set of mutations. this one contains everything which is not necessarily what we want


genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
 """)